In [17]:
pwd
set more off
clear all


C:\Users\wue04\Box Sync\fresh




### clean up raw distance output from ArcGIS

In [ ]:
forvalues i=12/16 {
    import delimited raw-output\closest-facility\closest_20`i'.csv, clear
    keep facilityrank incidentid total name 
    rename facilityrank rank
    rename total dist
    rename incidentid id
    split name, p(" - ")
    drop name name1
    rename name2 name
    reshape wide dist name, i(id) j(rank)
    gen year=20`i'
    compress
    save data\dist`i'.dta, replace
}
.

In [ ]:
forvalues i=12/15 {
    append using data\dist`i'.dta
    erase data\dist`i'.dta
}
.

merge 1:m id year using data\address\xy12-16.dta
tab year

In [ ]:
drop id _merge
order x y year
sort x y year
compress
save data\distance_xy_allyears.dta, replace
erase data\dist16.dta

### link xy distance to newid

In [ ]:
* clean up AP geocoded files
forvalues i=12/16 {
    use "S:\Restricted Data\Geocoding\AP\newid ap coordinates 20`i'.dta", clear
    keep newid year xcoord ycoord *boro* *zip* *atitude* *ongitude*
    rename xcoord x
    rename ycoord y
    rename *boro* boro
    rename *zip* zipcode
    rename *atitude* lat
    rename *ongitude* lon
    destring x y boro zipcode lat lon, replace
    compress
    save S:\Personal\hw1220\fresh\data\newid_xy_`i'.dta, replace
}
.

In [ ]:
forvalues i=12/15 {
    append using S:\Personal\hw1220\fresh\data\newid_xy_`i'.dta
    erase S:\Personal\hw1220\fresh\data\newid_xy_`i'.dta
}
.

In [52]:
tab year
duplicates tag newid year, gen(dup)
tab dup

merge m:1 x y year using data\distance_xy_allyears.dta

drop _merge dup
drop if missing(newid)|missing(year)
sort newid year

duplicates tag newid year, gen(dup)
tab dup
drop dup

drop if newid=="."
compress
save S:\Personal\hw1220\fresh\data\distance_newid.dta, replace
erase S:\Personal\hw1220\fresh\data\newid_xy_16.dta




Duplicates in terms of newid year


        dup |      Freq.     Percent        Cum.
------------+-----------------------------------
          0 |  5,591,788      100.00      100.00
------------+-----------------------------------
      Total |  5,591,788      100.00


### tab store coverage by year

In [62]:
tab year


Sheet1$_yea |
          r |      Freq.     Percent        Cum.
------------+-----------------------------------
       2012 |  1,054,760       18.86       18.86
       2013 |  1,093,100       19.55       38.41
       2014 |  1,145,820       20.49       58.90
       2015 |  1,164,178       20.82       79.72
       2016 |  1,133,929       20.28      100.00
------------+-----------------------------------
      Total |  5,591,787      100.00


In [67]:
bys year: tab name1 if dist1<=528 & (strmatch(name1, "*2050*")|strmatch(name1, "*305-325*")|strmatch(name1, "*3470*"))


---------------------------------------------------------------------------------------------------------------
-> year = 2012

                  1 name |      Freq.     Percent        Cum.
-------------------------+-----------------------------------
     2050 Webster Avenue |        248       40.33       40.33
        305-325 Avenue M |         96       15.61       55.93
       3470 Third Avenue |        271       44.07      100.00
-------------------------+-----------------------------------
                   Total |        615      100.00

---------------------------------------------------------------------------------------------------------------
-> year = 2013

                  1 name |      Freq.     Percent        Cum.
-------------------------+-----------------------------------
     2050 Webster Avenue |        244       37.89       37.89
        305-325 Avenue M |        104       16.15       54.04
       3470 Third Avenue |        296       45.96      100.00
----------

In [69]:
bys year: tab name2 if dist2<=528


---------------------------------------------------------------------------------------------------------------
-> year = 2012
no observations

---------------------------------------------------------------------------------------------------------------
-> year = 2013
no observations

---------------------------------------------------------------------------------------------------------------
-> year = 2014
no observations

---------------------------------------------------------------------------------------------------------------
-> year = 2015
no observations

---------------------------------------------------------------------------------------------------------------
-> year = 2016
no observations

